In [1]:
# %load_ext autotime
# import sys
# sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/magellan/')

This quickstart guide explains how to match two tables using Magellan. Our goal is to come up with a workflow to match DBLP and ACM datasets. Specifically, we want to achieve precision greater than 95% and get recall as high as possible. The datasets contain information about the conference papers published in top databse conferences. 

First, we need to import the Magellan package as follows:

In [2]:
# Import libraries
import magellan as mg
import pandas as pd
import os, sys

In [3]:
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + mg.__version__ )

python version: 2.7.12 |Continuum Analytics, Inc.| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]
pandas version: 0.18.1
magellan version: 0.1.0


Matching two tables typically consists of the following three steps:
1. Loading the input tables
2. Blocking the input tables to get a candidate set
3. Matching the tuple pairs in the candidate set

# 1. Loading the input tables

We begin by loading the input tables. For the purpose of this guide, we use the dataset that is included with the package.

In [4]:
dblp_dataset_path = os.sep.join(['DBLP_ACM', 'DBLP_cleaned.csv'])
acm_dataset_path = os.sep.join(['DBLP_ACM', 'ACM_cleaned.csv'])

In [5]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = mg.read_csv_metadata(dblp_dataset_path, key='id')
B = mg.read_csv_metadata(acm_dataset_path, key='id')

In [6]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2616
Number of tuples in B: 2294
Number of tuples in A X B (i.e the cartesian product): 6001104


In [7]:
A.head(2)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and its Application in Parallel-Join Load Balancing,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996


In [8]:
B.head(2)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999


In [9]:
# Display the key attributes of table A and B.
mg.get_key(A), mg.get_key(B)

('id', 'id')

# 2. Blocking to create candidate tuple pairs

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching. 

Magellan provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Refer to [api reference] for more details. The user can mix an match these blockers to form a blocking sequence 
applied to input tables.

For the matching problem at hand, we know that two conference papers published in different years cannot match. So, we decide to apply an attribute equivelance blocker on the 'year' attribute for this purpose. 

In [10]:
# Plan
# A, B ------ attribute equivalence [year] -----> C1

In [11]:
# Create attribute equivalence blocker
ab = mg.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
C1 = ab.block_tables(A, B, 'year', 'year', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

In [12]:
# Check the number of rows in C1
len(C1)

601284

In [13]:
# Display first two rows from C1
C1.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/sigmod/Mackay99,304586,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,The WASA2 object-oriented workflow management system,"Gottfried Vossen, Mathias Weske",1999
1,1,journals/sigmod/Mackay99,304587,Semantic Integration of Environmental Models for Application to Global Information Systems and D...,D. Scott Mackay,1999,A user-centered interface for querying distributed multimedia databases,"Isabel F. Cruz, Kimberly M. James",1999


The number of tuple pairs considered for matching is reduced to 601284 (from 6001104), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in Magellan as follows:

In [14]:
# Debug blocker output
dbg = mg.debug_blocker(C1, A, B, output_size=200)

In [15]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head()

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.875,journals/sigmod/Aberer02,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
1,1,0.875,journals/sigmod/Aberer02a,776994,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
2,2,0.875,journals/sigmod/Aberer02b,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
3,3,0.875,journals/sigmod/Aberer03b,601865,Book review column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record
4,4,0.875,journals/sigmod/Aberer02,604274,Book Review Column,Karl Aberer,SIGMOD Record,Book review column,Karl Aberer,ACM SIGMOD Record


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the conference papers to match the author names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [16]:
# Updated blocking sequence
# A, B ------ attribute equivalence [year] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [authors] --------> C2--

In [17]:
# Create an overlap blocker
ob = mg.OverlapBlocker()
# Apply overlap blocker on 'authors' attribute
C2 = ob.block_tables(A, B, 'authors', 'authors', 
                   l_output_attrs=['title', 'authors', 'year'],
                   r_output_attrs=['title', 'authors', 'year']
                   )

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [18]:
# Check the number of rows in C2
len(C2)

287414

In [19]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
0,0,journals/tods/LechtenborgerV03,304586,On the computation of relational view complements,vossen lechtenbrger jens gottfried,2003,The WASA2 object-oriented workflow management system,vossen mathias gottfried weske,1999
1,1,conf/sigmod/VossenW99,304586,The WASA2 Object-Oriented Workflow Management System,mathias vossen gottfried weske,1999,The WASA2 object-oriented workflow management system,vossen mathias gottfried weske,1999


In [20]:
# Combine blocker outputs
C = mg.combine_blocker_outputs_via_union([C1, C2])

In [21]:
# Check the number of rows in the consolidated candidate set.
len(C)

857777

We observe the number of tuple pairs considered for matching is increased to 875758 (from 601284). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [22]:
# Debug again
dbg = mg.debug_blocker(C, A, B)

In [23]:
# Display first few rows from the debugger output
dbg.head(3)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,rtable_title,rtable_authors,rtable_venue
0,0,0.500000,journals/sigmod/Dogac98,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
1,1,0.500000,journals/sigmod/Dogac02,945727,Guest Editor's Introduction,Asuman Dogac,SIGMOD Record,Guest editor's introduction,Karl Aberer,ACM SIGMOD Record
2,2,0.461538,journals/sigmod/Snodgrass98a,641001,Reminiscences on Influential Papers,Richard T. Snodgrass,SIGMOD Record,Reminiscences on influential papers,Kenneth A. Ross,ACM SIGMOD Record


We observe that the current blocker sequence does not drop potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.

# 3. Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set, obtained in the previous step. Specifically, we use learning-based method for matching purposes.

This typically involoves the following five steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## 3.1 Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [27]:
# Sample candidate set
S = mg.sample_table(C, 450)

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [28]:
# Label S and specify the attribute name for the label column
L = mg.label_table(S, 'gold')

For the purposes of this guide, we will load in a pre-labeled dataset included in this package.

In [29]:
# Load the pre-labeled data
L = mg.read_csv_metadata('DBLP_ACM/dblp_acm_demo_labels.csv', ltable=A, rtable=B)
# Display the number of rows in the labaled data set
len(L)

415

## 3.2 Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development and evaluation. Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data (i.e the evaluation set).

In [30]:
# Split the labeled data into development and evaluation set
development_evaluation = mg.split_train_test(L, train_proportion=0.7)
development =  development_evaluation['train']
evaluation = development_evaluation['test']

## 3.3 Select the best learning-based matcher

Selecting the best learning-based matcher typically involves the following steps:
1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

### 3.3.1 Creating a set of learning-based matchers

First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression. Refer [api ref] for more details. 

In [31]:
# Create a set of ML-matchers
dt = mg.DTMatcher(name='DecisionTree')
svm = mg.SVMMatcher(name='SVM')
rf = mg.RFMatcher(name='RF')
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

### 3.3.2 Creating features

Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the statistical characteristics of the input tables. For the purpose of this guide, we use the automatically generated features.

In [32]:
# Generate features
feature_table = mg.get_features_for_matching(A, B)

In [33]:
# List the names of the features generated
feature_table['feature_name']

0             title_title_jac_qgm_3_qgm_3
1         title_title_cos_dlm_dc0_dlm_dc0
2                         title_title_mel
3                    title_title_lev_dist
4                     title_title_lev_sim
5         authors_authors_jac_qgm_3_qgm_3
6     authors_authors_cos_dlm_dc0_dlm_dc0
7                     authors_authors_mel
8                authors_authors_lev_dist
9                 authors_authors_lev_sim
10                          year_year_exm
11                          year_year_anm
12                     year_year_lev_dist
13                      year_year_lev_sim
Name: feature_name, dtype: object

We observe that there were 14 features generated. As a first step, lets say that we decide to use only year related features.

In [34]:
# Select the author related features
feature_subset_iter1 = feature_table[10:14]

In [35]:
# List the names of the features selected
feature_subset_iter1['feature_name']

10         year_year_exm
11         year_year_anm
12    year_year_lev_dist
13     year_year_lev_sim
Name: feature_name, dtype: object

### 3.3.3 Extracting feature vectors

In this step, we extract feature vectors using the development set and the created features.

In [36]:
# Extract feature vectors
feature_vectors_dev = mg.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='gold')                                         

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [37]:
# Display first few rows
feature_vectors_dev.head(3)

,_id,ltable_id,rtable_id,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,gold
163,163,conf/vldb/ChaudhuriW00,671696,1,1.0,0.0,1.0,1
12,12,conf/sigmod/BerchtoldK98,276353,1,1.0,0.0,1.0,1
276,276,journals/sigmod/CheungF98,290596,1,1.0,0.0,1.0,1


Next, we may have to impute the feature vectors as it may contain missing values. First, let us check if there are any missing values in the extracted feature vectors.

In [38]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_dev))

True

We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column. 

In [39]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = mg.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

### 3.3.4 Selecting the best matcher using cross-validation

Now, we select the best matcher using cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [40]:
# Select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='precision') 

In [41]:
# Check the cross validation statistics
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x000000000E94ABA8>,5,0.772727,0.804348,0.767442,0.692308,0.756757,0.758716
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x000000000E94AC88>,5,0.750000,0.813953,0.690476,0.888889,0.681818,0.765027
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x000000000E94ABE0>,5,0.729167,0.702703,0.800000,0.750000,0.825000,0.761374
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x000000000E94ACF8>,5,0.744681,0.731707,0.883721,0.769231,0.666667,0.759201
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x000000000E94ADA0>,5,0.634146,0.731707,0.818182,0.738095,0.878049,0.760036
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x000000000E94AE10>,5,0.659091,0.711111,0.861111,0.853659,0.744186,0.765832


### 3.3.5 Debugging matcher

We observe that the best matcher is not getting us to the precision that we expect (i.e > 95%). We debug the matcher to see what might be wrong.

To do this, first we split the feature vectors into train and test.

In [42]:
# # Split feature vectors into train and test
train_test = mg.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher debugging purposes.

In [43]:
# Debug decision tree using GUI
mg.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold')

From the GUI, we observe that using only year related features result in a lot of false positives. So we decide to all features in the feature table, which had author, title and year related features.

In [44]:
# Select all features from the feature table
feature_subset_iter2 = feature_table

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [45]:
# Get new set of features
feature_vectors_dev = mg.extract_feature_vecs(development, feature_table=feature_subset_iter2, attrs_after='gold')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [46]:
# Check if imputation is required
any(pd.isnull(feature_vectors_dev))

True

In [47]:
# Impute feature vectors
feature_vectors_dev = mg.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

In [49]:
# Apply cross validation to find if there is a better matcher
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold', metric='f1') 

In [50]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object at 0x000000000E94ABA8>,5,1.000000,0.949153,0.984127,1.000000,0.969697,0.980595
1,RF,<magellan.matcher.rfmatcher.RFMatcher object at 0x000000000E94AC88>,5,0.986301,1.000000,0.985075,1.000000,0.983607,0.990997
2,SVM,<magellan.matcher.svmmatcher.SVMMatcher object at 0x000000000E94ABE0>,5,1.000000,0.816327,0.929577,0.935484,0.937500,0.923778
3,NB,<magellan.matcher.nbmatcher.NBMatcher object at 0x000000000E94ACF8>,5,0.983051,0.981818,0.983051,1.000000,1.000000,0.989584
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher object at 0x000000000E94ADA0>,5,0.967742,1.000000,0.966667,0.949153,1.000000,0.976712
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher object at 0x000000000E94AE10>,5,0.966667,1.000000,0.967742,1.000000,0.966667,0.980215


Now, observe the best matcher is achieving the expected precision and we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## 3.4 Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Extracting the feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### 3.4.1 Extracting the feature vectors

As before, we extract the feature vectors (using the updated feature table and the evaluation set) and impute it (if necessary).

In [51]:
# Get new set of features
feature_vectors_eval = mg.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter2, 
                                               attrs_after='gold')


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [52]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_eval))

True

In [53]:
# Impute feature vectors
feature_vectors_eval = mg.impute_table(feature_vectors_eval, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
                strategy='mean')

### 3.4.2 Training the matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [54]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors_dev, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')


### 3.4.3 Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [55]:
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

### 3.4.4 Evaluating the matching output

Finally, we evaluate the predicted outputs

In [57]:
# Evaluate the result
eval_result = mg.eval_matches(predictions, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 98.61% (71/72)
Recall : 98.61% (71/72)
F1 : 98.61%
False positives : 1 (out of 72 positive predictions)
False negatives : 1 (out of 53 negative predictions)
